# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

Импорт библиотек

In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Загрузка данных

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

Функция для первичного анализа данных

In [3]:
def df_info(ds):
    print(50*"-")
    print('Общая информация:')
    print(ds.info())
    print(50*"-")
    print('Размер:')
    print(ds.shape)
    print(60*"-")
    print('Пример данных в начале:')
    print(ds.head())
    print(60*"-")
    print('Пример данных в конце:')
    print(ds.tail())
    print(78*"-")
    print('Описание:')
    print(ds.describe())
    print(78*"-")
    print('Дубликаты:')
    print(ds.duplicated().sum())

In [4]:
df_info(df)

--------------------------------------------------
Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
--------------------------------------------------
Размер:
(5000, 5)
------------------------------------------------------------
Пример данных в начале:
   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2        

Вывод примера дубликатов

In [5]:
df.loc[df.duplicated(keep=False)].sort_values(by='Зарплата').head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32.0,21600.0,0,0
2988,1,32.0,21600.0,0,0
361,0,50.0,24700.0,1,2
2869,0,50.0,24700.0,1,2
333,0,32.0,25600.0,1,0
4230,0,32.0,25600.0,1,0
1378,0,36.0,26400.0,0,0
2723,0,36.0,26400.0,0,0
1002,1,34.0,26900.0,0,0
1140,1,34.0,26900.0,0,0


Анализ уникальных значений

In [6]:
cols = ['Пол', 'Возраст',  'Зарплата',  'Члены семьи',  'Страховые выплаты']
print('Количество уникальных значений по столбцам')
for i in cols:
    print('%s:' %i, df[i].nunique())

Количество уникальных значений по столбцам
Пол: 2
Возраст: 46
Зарплата: 524
Члены семьи: 7
Страховые выплаты: 6


<b> ВЫВОДЫ </b>

 - данные загружены;
 - обнаружено 153 дубликата: т.к. данные малоинформативны и количество дубликатов мало, стоит их оставить;
 - судя по полченному описанию (describe) данные хорошие, пропусков и выбросов нет.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

<b> Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)<br>
 - a. Изменится. Приведите примеры матриц.
 - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Обоснование:**

Пусть новая матрица признаков $X_1$ как произведение старой $X$ на матрицу $P$(обратимую):

$$X_1=XP$$

Применим значение $X_1$ к формуле обучения:
$$w_1 = ((XP)^T XP)^{-1} (XP)^T y$$

Раскроем скобки, выделив $P$:
$$w_1 = (X^TP^T XP)^{-1} X^TP^T y$$

$$w_1 = (X^T X)^{-1} X^T P^{-1} (P^T)^{-1} P^Ty$$

Так как $P$ обратимая, то произведение $(P^T)^{-1}P^T$ равно единичной матрице($E$):
$$w_1 = P^{-1} (X^T X)^{-1} X^T y$$

Видим, что справа получилась формула для $w$:
$$w_1 = P^{-1} w$$

Воспользовавшись формулой предсказаний линейной регрессии, получаем:
$$a_1 = X_1 w_1$$

$$a_1 = X P P^{-1} w$$

$$a_1 = X E w$$

$$a_1 = X w = a$$

<b> Мы доказали, что предсказания для матрицы признаков ($a_1$), умноженных на обратимую матрицу ($P$) равны предсказаниям $a$.
    
В данном случае, связь параметров линейной регрессии в исходной задаче $w$ и в преобразованной $w_1$ выглядит так:
$$w_1 = P^{-1} w$$

## Алгоритм преобразования

**Алгоритм**

В качестве алгоритма преобразования используем матрицу со случайными величинами, у которой существует обратная к ней матрица.

**Обоснование**

In [7]:
#Разделение данных на целевые (target) и признаки (features):
target = df.loc[:, 'Страховые выплаты']
features = df.drop(['Страховые выплаты'], axis=1)

In [8]:
#Разделение данных на тренировочную (train) и тестовую (test) выборки:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

In [9]:
#Преобразование выборкок в матрицы:
m_ftr = features_train.values
m_ft = features_test.values
print(m_ftr.shape)
print(m_ft.shape)

(3750, 4)
(1250, 4)


In [10]:
#Создадим рандомную обратимую матрицу 4 на 4 (по количеству признаков основной матрицы):

rand_m = np.random.randn(4, 4)
rand_m = np.round(rand_m)
rand_m

array([[ 1., -1.,  1.,  1.],
       [-1.,  2.,  2., -0.],
       [ 2.,  1., -1.,  1.],
       [ 0., -0.,  1., -0.]])

In [11]:
#Проверим получившейся матрицы на обратимость.
np.round(rand_m @ np.linalg.inv(rand_m))

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1., -0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0., -0.,  1.]])

В результате получилась единичная матрица - МАТРИЦА ОБРАТИМА


In [12]:
#Перемножим тестовую и тренировочную матрицы на получившуюся рандомную обратимую матрицу:
с_m_ftr = m_ftr.dot(rand_m)
с_m_ft = m_ft.dot(rand_m)

Для восстановления исходных данных перемножим Тренировочные признаки (train) на обратную рандомной матрицу:

In [13]:
dc_m_ftr = с_m_ftr.dot(np.linalg.inv(rand_m))

Восстановим DataFrame из матрицы:

In [14]:
f_tr_check = pd.DataFrame(dc_m_ftr, columns=features_train.columns)
f_tr_check, features_train

(      Пол  Возраст  Зарплата  Члены семьи
 0     1.0     43.0   36200.0          1.0
 1     1.0     34.0   57600.0          0.0
 2     0.0     32.0   41100.0          1.0
 3     0.0     36.0   45100.0          1.0
 4     0.0     33.0   50600.0          2.0
 ...   ...      ...       ...          ...
 3745  0.0     42.0   32100.0          0.0
 3746  0.0     28.0   22700.0          4.0
 3747  1.0     41.0   44700.0          1.0
 3748  0.0     22.0   50100.0          4.0
 3749  0.0     19.0   40800.0          0.0
 
 [3750 rows x 4 columns],
       Пол  Возраст  Зарплата  Члены семьи
 3369    1     43.0   36200.0            1
 1441    1     34.0   57600.0            0
 571     0     32.0   41100.0            1
 225     0     36.0   45100.0            1
 2558    0     33.0   50600.0            2
 ...   ...      ...       ...          ...
 3497    0     42.0   32100.0            0
 3492    0     28.0   22700.0            4
 2177    1     41.0   44700.0            1
 3557    0     22.0   5010

<b> Данные не изменились

## Проверка алгоритма

Создадим модель для матриц с помощью Линейной регрессии

In [15]:
model = LinearRegression()
model.fit(m_ftr, target_train)
predicted_norm = model.predict(features_test)
norm_r2 = model.score(features_test, target_test).round(10)
print('Оценка R2 для некодированной матрицы: ', norm_r2)

Оценка R2 для некодированной матрицы:  0.4352275713


In [16]:
model.fit(с_m_ftr, target_train)
predicted_с = model.predict(с_m_ft)
coded_r2 = model.score(с_m_ft, target_test).round(10)
print('Оценка R2 для кодированной матрицы: ', coded_r2)

Оценка R2 для кодированной матрицы:  0.4352275713


## Вывод

Проверив показатели качества моделей Линейной регрессии с помощью метрики R2 на данных (сначала на исходных, затем умноженных на обратимую матрицу) размер которой равен числу признаков. 

Показатели метрики совпадают, следовательно можно утверждать, что алгоритм работает.